# Least Squares Adjustment for Quadrilateral Coordinates
# ====================================================
# This script computes the coordinates of four points (P1, P2, P3, P4) forming a quadrilateral
# using least squares adjustment, based on observed distances, azimuths, and height differences
# from three fixed stations.

In [1]:
import numpy as np

# Step 1: Define Input Data
# -------------------------

## Fixed station coordinates (in meters)

In [2]:

stations = {
    'P1': {'X': 356265.967, 'Y': 5787646.936, 'Z': 87.432},
    'P2': {'X': 356262.534, 'Y': 5787647.786, 'Z': 87.386},
    'P3': {'X': 356252.905, 'Y': 5787649.079, 'Z': 87.289}
}

## Approximate coordinates of unknown points (P1, P2, P3, P4)

In [3]:

approx_coords = {
    1: {'X': 356268.0, 'Y': 5787660.0, 'Z': 87.0},
    2: {'X': 356268.0, 'Y': 5787660.0, 'Z': 96.0},
    3: {'X': 356262.0, 'Y': 5787662.0, 'Z': 96.0},
    4: {'X': 356262.0, 'Y': 5787662.0, 'Z': 87.0}
}


## Observations

In [4]:

distances = [
    ('P1', 1, 13.232), ('P1', 2, 15.811), ('P1', 3, 17.867),
    ('P2', 1, 13.390), ('P2', 2, 15.970), ('P2', 3, 16.717), ('P2', 4, 14.239),
    ('P4', 3, 18.046), ('P4', 4, 15.860)
]
azimuths = [
    ('P1', 1, 9.22789), ('P1', 2, 8.95359), ('P1', 3, 345.45717),
    ('P2', 1, 24.49167), ('P2', 2, 24.21737), ('P2', 3, 358.05866), ('P2', 4, 357.90458),
    ('P4', 3, 35.52366), ('P4', 4, 35.24937)
]
height_diffs = [
    ('P1', 1, 0.43), ('P4', 4, 0.29)
]


## Observation precisions

In [5]:
sigma_d = 0.01  # meters
sigma_az = 0.005  # radians
sigma_h = 0.01  # meters


## Weights

In [6]:
w_d = 1 / sigma_d**2  # 10000 m^-2
w_az = 1 / sigma_az**2  # 40000 rad^-2
w_h = 1 / sigma_h**2  # 10000 m^-2


## Observed values (convert azimuths to radians)

In [7]:
obs_d = np.array([d[2] for d in distances])
obs_az = np.radians(np.array([az[2] for az in azimuths]))
obs_h = np.array([h[2] for h in height_diffs])
l_obs = np.hstack([obs_d, obs_az, obs_h])


## Weight matrix

In [8]:
weights = [w_d] * len(distances) + [w_az] * len(azimuths) + [w_h] * len(height_diffs)
W = np.diag(weights)

# Step 2: Initialize Parameters
# -----------------------------

## Unknowns: [X1, Y1, Z1, X2, Y2, Z2, X3, Y3, Z3, X4, Y4, Z4]


In [9]:
X = np.array([
    approx_coords[1]['X'], approx_coords[1]['Y'], approx_coords[1]['Z'],
    approx_coords[2]['X'], approx_coords[2]['Y'], approx_coords[2]['Z'],
    approx_coords[3]['X'], approx_coords[3]['Y'], approx_coords[3]['Z'],
    approx_coords[4]['X'], approx_coords[4]['Y'], approx_coords[4]['Z']
])

# Step 3: Least Squares Adjustment
# --------------------------------

In [10]:
def compute_observations(X, distances, azimuths, height_diffs, stations):
    """Compute calculated observations and Jacobian matrix."""
    l_calc = []
    A = np.zeros((len(distances) + len(azimuths) + len(height_diffs), len(X)))
    
    # Distances
    for idx, (s, p, _) in enumerate(distances):
        Xs, Ys = stations[s]['X'], stations[s]['Y']
        Xp, Yp, Zp = X[3*(p-1)], X[3*(p-1)+1], X[3*(p-1)+2]
        d = np.sqrt((Xp - Xs)**2 + (Yp - Ys)**2 + (Zp - stations[s]['Z'])**2)
        l_calc.append(d)
        # Partial derivatives
        A[idx, 3*(p-1)] = (Xp - Xs) / d  # dD/dX
        A[idx, 3*(p-1)+1] = (Yp - Ys) / d  # dD/dY
        A[idx, 3*(p-1)+2] = (Zp - stations[s]['Z']) / d  # dD/dZ
    
    # Azimuths
    for idx, (s, p, _) in enumerate(azimuths, len(distances)):
        Xs, Ys = stations[s]['X'], stations[s]['Y']
        Xp, Yp = X[3*(p-1)], X[3*(p-1)+1]
        dx, dy = Xp - Xs, Yp - Ys
        az = np.arctan2(dy, dx)
        l_calc.append(az)
        # Partial derivatives
        den = dx**2 + dy**2
        A[idx, 3*(p-1)] = -dy / den  # dAz/dX
        A[idx, 3*(p-1)+1] = dx / den  # dAz/dY
    
    # Height differences
    for idx, (s, p, _) in enumerate(height_diffs, len(distances) + len(azimuths)):
        Zp = X[3*(p-1)+2]
        Zs = stations[s]['Z']
        l_calc.append(Zp - Zs)
        A[idx, 3*(p-1)+2] = 1  # dH/dZ
    
    return np.array(l_calc), A


## Iteration

In [11]:

it = 0
dx = np.ones(len(X))
threshold = 1e-4  # Convergence threshold (meters)

while np.max(np.abs(dx)) >= threshold:
    it += 1
    l_calc, A = compute_observations(X, distances, azimuths, height_diffs, stations)
    
    # Azimuth observation adjustment: convert observed to coordinate system
    l_obs_adjusted = l_obs.copy()
    for i in range(len(distances), len(distances) + len(azimuths)):
        l_obs_adjusted[i] = np.radians(azimuths[i - len(distances)][2] - 90)
    
    # Residuals
    l = l_obs_adjusted - l_calc
    
    # Normal equations
    N = A.T @ W @ A
    U = A.T @ W @ l
    dx = np.linalg.solve(N, U)
    
    # Update coordinates
    X += dx


KeyError: 'P4'

# Step 4: Compute Covariance and Residuals
# ---------------------------------------

In [12]:

l_calc, A = compute_observations(X, distances, azimuths, height_diffs, stations)
l_obs_adjusted = l_obs.copy()
for i in range(len(distances), len(distances) + len(azimuths)):
    l_obs_adjusted[i] = np.radians(azimuths[i - len(distances)][2] - 90)
v = l_obs_adjusted - l_calc

# Variance of unit weight
n = len(l_obs)
u = len(X)
sigma_0_squared = (v.T @ W @ v) / (n - u)

# Covariance matrix
cov_matrix = sigma_0_squared * np.linalg.inv(A.T @ W @ A)
sigma_x = np.sqrt(np.diag(cov_matrix))


KeyError: 'P4'

# Step 5: Display Results
# -----------------------

In [13]:
print(f"Number of iterations: {it}")
print(f"\nFinal Coordinates:")
for i in range(0, len(X), 3):
    print(f"P{i//3 + 1}: X={X[i]:.3f} m, Y={X[i+1]:.3f} m, Z={X[i+2]:.3f} m")

print(f"\nStandard Deviations:")
for i in range(0, len(X), 3):
    print(f"P{i//3 + 1}: σ_X={sigma_x[i]:.3f} m, σ_Y={sigma_x[i+1]:.3f} m, σ_Z={sigma_x[i+2]:.3f} m")

print(f"\nVariance of unit weight (σ_0²): {sigma_0_squared:.4f}")
print(f"\nResiduals:")
print("Distances (m):", v[:len(distances)])
print("Azimuths (rad):", v[len(distances):len(distances) + len(azimuths)])
print("Height Differences (m):", v[len(distances) + len(azimuths):])

Number of iterations: 1

Final Coordinates:
P1: X=356268.000 m, Y=5787660.000 m, Z=87.000 m
P2: X=356268.000 m, Y=5787660.000 m, Z=96.000 m
P3: X=356262.000 m, Y=5787662.000 m, Z=96.000 m
P4: X=356262.000 m, Y=5787662.000 m, Z=87.000 m

Standard Deviations:


NameError: name 'sigma_x' is not defined